In [2]:
import nibabel as nib
import numpy as np
import pandas as pd
from shapely import geometry
import geopandas as gpd

In [7]:
#Calculate the most frequently-occurring value in each row of a 2D NumPy arra
def bincount2D_vectorized(a):
    N = a.max()+1
    a_offs = a + np.arange(a.shape[0])[:,None]*N
    return np.bincount(a_offs.ravel(), minlength=a.shape[0]*N).reshape(-1,N)

In [8]:
#Search for values in a list that exist in a 2D NumPy array
def isin2d(arr1,arr2):
    return (arr1==arr2[:,None]).all(2).any(0)

In [9]:
#Smooth a vector by inserting the mid - point between every two adjacent points
def smooth_border(vertex_input,coordinate_input):
    coordinate_output = []
    for i in range(0,len(vertex_input)):
        if np.isin(vertex_input[i],cut_vertex) or np.isin(vertex_input[i],more_vertex):
        #if np.isin(vertex_input[i],more_vertex) or :
            coordinate_output = coordinate_output + [coordinate_input[i]]
        else:
            coordinate_output = coordinate_output + [(coordinate_input[i]+coordinate_input[i-1])/2]
    return np.array(coordinate_output)

In [6]:
def sort_border(edge_def,flatmap_list_def_input):
    flatmap_list_def = flatmap_list_def_input.copy()
    border_def = np.array([edge_def[0]])
    multi_plot = False
    vertex = [border_def[0,0]]
    
    for i in range(1,len(edge_def)):
        last_edge_def = border_def[-1]
        next_edge_def = edge_def[(np.isin(edge_def[:,0],last_edge_def)|np.isin(edge_def[:,1],last_edge_def))&(~isin2d(edge_def,border_def))&
        ((edge_def[:,0]==vertex[-1])|(edge_def[:,1]==vertex[-1]))]
        if len(next_edge_def)==0:
            multi_plot = True
            break
        border_def = np.concatenate([border_def,np.array([next_edge_def[0]])])
        vertex = vertex + [border_def[-1][border_def[-1]!=vertex[-1]][0]]

    if len(vertex)>2:
        flatmap_list_def = flatmap_list_def + [[smooth_border(vertex,flatmap[vertex])]]
    if multi_plot:
        edge_def = edge_def[~isin2d(edge_def,border_def)]
        flatmap_list_def = sort_border(edge_def,flatmap_list_def)
    return flatmap_list_def

In [256]:
#Create a GeoDataFrame
#Input mesh: n x 3 array. n is triangle count, 3 are flatmap coordinate indices.
#Input flatmap: m x 3 array. m is vertex count, 3 are vertex coords (x, y, z with z usually 0).
#Input label: m - length array. Labels each vertex numerically.
#Input label_list & rename_list: Used for label renaming & merging. E.g., [1,2,3] and ["A","A","B"] merge 1,2 to "A" and rename 3 to "B".
def create_GeoDataFrame(mesh,flatmap,label,label_list,rename_list):
    cut_edge = pd.DataFrame(np.sort(np.concatenate([mesh[:,[0,1]],mesh[:,[1,2]],mesh[:,[0,2]]])),columns=['v1','v2'])
    cut_edge['edge'] = cut_edge['v1'].map(str) + '_' + cut_edge['v2'].map(str)
    #Retrieve the distinct edges that form the boundary of the mesh
    cut_edge = cut_edge.drop_duplicates(subset=['edge'],keep=False)
    cut_edge = np.array(cut_edge[['v1','v2']])
    cut_vertex = np.unique(cut_edge.flatten())

    mesh_label = label[mesh]
    mesh_label = np.argmax(bincount2D_vectorized(mesh_label),axis=1)
    all_edge = []
    for n in np.unique(label):
        n_mesh = mesh[mesh_label==n]
        if len(n_mesh)==0:
            continue
        if n==0:
            continue
        edge = pd.DataFrame(np.sort(np.concatenate([n_mesh[:,[0,1]],n_mesh[:,[1,2]],n_mesh[:,[0,2]]])),columns=['v1','v2'])
        edge['edge'] = edge['v1'].map(str) + '_' + edge['v2'].map(str)
        edge = edge.drop_duplicates(subset=['edge'],keep=False)
        all_edge = all_edge + [edge]
        
    all_edge = pd.concat(all_edge)
    all_vertex = np.array(all_edge[['v1','v2']]).flatten()
    more_vertex = np.arange(0,max(all_vertex)+1)[np.bincount(all_vertex)>=6]
    
    mesh_label = label[mesh]
    mesh_label = np.argmax(bincount2D_vectorized(mesh_label),axis=1)

    Polygon_list = []
    region_index = []
    region_label = pd.DataFrame({'Index':label_list,'new_name':rename_list})
    for n in pd.unique(region_label['new_name']):
        n_mesh = mesh[np.isin(mesh_label,np.array(region_label[region_label['new_name']==n]['Index']))]
        if len(n_mesh)==0:
            continue
        edge = pd.DataFrame(np.sort(np.concatenate([n_mesh[:,[0,1]],n_mesh[:,[1,2]],n_mesh[:,[0,2]]])),columns=['v1','v2'])
        edge['edge'] = edge['v1'].map(str) + '_' + edge['v2'].map(str)
        edge = edge.drop_duplicates(subset=['edge'],keep=False)
        edge = np.array(edge[['v1','v2']])
        flatmap_list = []
        flatmap_list = sort_border(edge,flatmap_list)
        if len(flatmap_list)==1:
            Polygon_list = Polygon_list + [geometry.Polygon(flatmap_list[0][0])]
        else:
            Polygon_list = Polygon_list + [geometry.MultiPolygon(flatmap_list)] 
        region_index = region_index + [n]

    cq = gpd.GeoSeries(Polygon_list,
                     crs='EPSG:4326')
    df = pd.DataFrame({'region':region_index})
    df = gpd.GeoDataFrame(df,geometry=cq)
    return(df)

<font size=10>---------------Marmoset flatmap-------------<font size=10>

In [268]:
#marmoset MRI flatmap data, download from https://db.cngb.org/stomics/mbcsta/download/ or https://marmosetbrainmapping.org/data.html#alldata
flatmap = nib.load("data/MRI_raw_data/marmoset/surfFS.rh.full.flat.patch.surf.gii")
label = nib.load("data/MRI_raw_data/marmoset/surfFS.rh.MBM_cortex_vPaxinos.label.gii")
region_name = pd.read_csv('data/MRI_raw_data/marmoset/atlas_MBM_cortex_vPaxinos.txt',sep='\s+',header=None)
region_name = region_name[region_name[1]!='background']
VL_change = pd.read_csv('Tabledata/All2VL.txt',sep='\s+')
VL_change = VL_change[VL_change['species']=='Marmoset']
VL_change.index = VL_change['region']

In [269]:
#get the array from the nib object
mesh = flatmap.darrays[1].data
flatmap = flatmap.darrays[0].data[:,0:2]
label = np.round(label.darrays[0].data,0)

In [270]:
#horizontal mirror flatmap coordiante
mirror_matrix = np.array([[-1, 0], [0, 1]])
flatmap = np.dot(flatmap, mirror_matrix.T)
#rotation flatmap coordiante 75°
theta = np.radians(75)
rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                            [np.sin(theta), np.cos(theta)]])
flatmap = np.dot(flatmap, rotation_matrix.T)

In [271]:
#create the marmoset geodataframe
df = create_GeoDataFrame(mesh,flatmap,label,region_name[0],region_name[1])

In [272]:
#add uniform_region label to flatmap
df['VL'] = [VL_change.loc[i]['Uniform_region'] for i in df['region'].tolist()]

In [273]:
#save as shp file, pre-processed files can be downloaded from https://db.cngb.org/stomics/mbcsta/download/ or https://github.com/haoshijie13/MCCSTA/tree/main/Stereo_analysis
df.to_file('data/MRI_shp_file/Marmoset.shp')

<font size=10>---------------Macaque flatmap-------------<font size=10>

In [ ]:
from sklearn.neighbors import KDTree

In [153]:
#macaque MRI flatmap data, download from https://db.cngb.org/stomics/mbcsta/download/ or https://www.scidb.cn/en/detail?dataSetId=f6eead10c2f84e9d91951cee2837048f
flatmap = nib.load("data/MRI_raw_data/macaque/civm.L.flat.164k_fs_LR.surf.gii")
label = nib.load("data/MRI_raw_data/macaque/d1_D99_atlas_in_BNA_L2.shape.gii")
region_name = pd.read_csv("data/MRI_raw_data/macaque/region_label.txt",sep='\t',header=0)
VL_change = pd.read_csv('Tabledata/All2VL.txt',sep='\s+')
VL_change = VL_change[VL_change['species']=='Macaque']
VL_change.index = VL_change['region']

In [119]:
mesh = flatmap.darrays[1].data
flatmap = flatmap.darrays[0].data[:,0:2]
#remove float point
label = np.round(label.darrays[0].data,0)

#imputation the NA value in label by 10 nearest point
ref_label = label[np.isin(label,region_name['Index'])].astype('int')
ref_flatmap = flatmap[np.isin(label,region_name['Index'])]
kdt = KDTree(ref_flatmap, leaf_size=30, metric='euclidean')
result = kdt.query(flatmap,k=10, return_distance=False)
for i in range(0,2):
    query_label = np.argmax(bincount2D_vectorized(ref_label[result]),axis=1)
    label = query_label.copy().astype('int')

In [120]:
#create the marmoset geodataframe
df = create_GeoDataFrame(mesh,flatmap,label,region_name['Index'],region_name['new_name'])

In [124]:
#add uniform_region label to flatmap
df['VL'] = [VL_change.loc[i.split('-')[0]]['Uniform_region'] for i in df['region'].tolist()]

In [125]:
#save as shp file, pre-processed files can be downloaded from https://db.cngb.org/stomics/mbcsta/download/ or https://github.com/haoshijie13/MCCSTA/tree/main/Stereo_analysis
df.to_file('data/MRI_shp_file/Macaque.shp')

<font size=10>---------------Human flatmap-------------<font size=10>

In [141]:
#human MRI flatmap data, download from https://db.cngb.org/stomics/mbcsta/download/ or https://balsa.wustl.edu/WN56
flatmap = nib.load("data/MRI_raw_data/human/Q1-Q6_RelatedParcellation210.L.flat.32k_fs_LR.surf.gii")
label_raw = nib.load("data/MRI_raw_data/human/Q1-Q6_RelatedParcellation210.L.CorticalAreas_dil_Colors.32k_fs_LR.dlabel.nii")
region_name = pd.read_csv("data/MRI_raw_data/human/region_list.txt",sep='\t',header=None)
VL_change = pd.read_csv('Tabledata/All2VL.txt',sep='\s+')
VL_change = VL_change[VL_change['species']=='Human']
VL_change.index = VL_change['region']

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [136]:
mesh = flatmap.darrays[1].data
flatmap = flatmap.darrays[0].data[:,0:2]
#get human label from nib object
label = np.zeros((label_raw.header.get_axis(1).vertex.max()+1),dtype=int)
label[label_raw.header.get_axis(1).vertex] = label_raw.get_fdata()[0]

In [139]:
#create the marmoset geodataframe
df = create_GeoDataFrame(mesh,flatmap,label,region_name[0],region_name[1])

In [142]:
#add uniform_region label to flatmap
df['VL'] = [VL_change.loc[i]['Uniform_region'] for i in df['region'].tolist()]

In [144]:
#save as shp file, pre-processed files can be downloaded from https://db.cngb.org/stomics/mbcsta/download/ or https://github.com/haoshijie13/MCCSTA/tree/main/Stereo_analysis
df.to_file('data/MRI_shp_file/Human.shp')

<font size=10>---------------Mouse flatmap-------------<font size=10>

In [145]:
import cv2
from skimage import io

In [147]:
#human MRI flatmap data, download from https://db.cngb.org/stomics/mbcsta/download/
img = io.imread('data/MRI_raw_data/mouse/flatmap_dorsal.tif')
region_label = pd.read_csv('data/MRI_raw_data/mouse/region_name.txt',sep='\t',header=0)

In [148]:
poly_list = []
for label in region_label['id']:
    region_img = img.copy()
    region_img[region_img!=label] = 0
    region_img[region_img==label] = 255
    region_img = region_img.astype('uint8')
    contours, hierarchy = cv2.findContours(region_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE) 
    if(len(contours)>1):
        multipoly = []
        for i in range(0,len(contours)):
            if contours[i].shape[0] < 4:
                continue
            multipoly = multipoly + [geometry.Polygon(-np.reshape(newshape=(len(contours[i]),2),a=contours[i]))]
        multipoly = geometry.MultiPolygon(multipoly)
        poly_list = poly_list + [multipoly]
    else:
        singlepoly = geometry.Polygon(-np.reshape(newshape=(len(contours[0]),2),a=contours[0]))
        poly_list = poly_list + [singlepoly]

In [149]:
cq = gpd.GeoSeries(poly_list)

In [150]:
df = gpd.GeoDataFrame(region_label['id'],geometry=cq)
df[['name','region']] = region_label[['name','region']]

In [151]:
#save as shp file, pre-processed files can be downloaded from https://db.cngb.org/stomics/mbcsta/download/ or https://github.com/haoshijie13/MCCSTA/tree/main/Stereo_analysis
df.to_file('data/MRI_shp_file/Mouse.shp')

/mnt/SSD4Ta/home/huangzhi/anaconda3/envs/work/lib/python3.11/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
